In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [ ]:
# 1. Load Libraries and Utilities
from langchain_groq import ChatGroq
from teammerge.utils import load_esco_data, calculate_skill_gap
from teammerge.tools import get_skill_tool, get_occupation_tool, get_skill_gap_tool, make_skill_lookup
from teammerge.agents import build_hr_agent, build_esco_agent, build_planner_agent

/home/ec2-user/environment/teammerge-esco-agent/teammerge/agents.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
# 2. Load ESCO Data
skills_df, occupations_df = load_esco_data("data/esco/skills_en.csv", "data/esco/occupations_en.csv")

KeyError: "['hasEssentialSkill', 'hasOptionalSkill'] not in index"

In [ ]:
# 3. Build Lookup Function 
skill_lookup = make_skill_lookup(skills_df)

In [ ]:
# 4. Load ChromaDB Vectorstores
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

skill_vectorstore = Chroma(persist_directory="./chroma/skills_index", embedding_function=embedding_model)
occupation_vectorstore = Chroma(persist_directory="./chroma/occupations_index", embedding_function=embedding_model)

skill_vectorstore.persist()
occupation_vectorstore.persist()

In [ ]:
# 6. Initialize LLM
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [ ]:
# 7. Create Agents
hr_agent = build_hr_agent(llm)
esco_agent = build_esco_agent(llm)
planner_agent = build_planner_agent(llm)

In [ ]:
# 8. Routing Logic

def route_to_agent(user_input: str):
    if "team" in user_input or "restructure" in user_input or "manager" in user_input:
        return hr_agent.run(user_input)
    elif "skill" in user_input or "ESCO" in user_input or "occupation" in user_input:
        return esco_agent.run(user_input)
    elif "plan" in user_input or "transition" in user_input or "reskill" in user_input:
        return planner_agent.run(user_input)
    else:
        return esco_agent.run(user_input)

In [ ]:
# 9. Interactive Loop (optional in notebook)
print("🟢 TeamMerge Agent Ready. Type 'exit' to stop.\n")

while True:
    user_input = input("👤 You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    print("🤖 Agent:\n" + route_to_agent(user_input) + "\n")